In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

from datetime import date

import matplotlib.pyplot as plt
%matplotlib inline
import sys
from sklearn.feature_extraction import DictVectorizer

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy

from xgboost import XGBRegressor

In [2]:
num_f = ['model_age', 'm_from_car_reg', 'power_kw', 'mileage_in_km']
cat_f = ['brand', 'model', 'color', 'transmission_type', 'fuel_type']
target = ['price_in_euro']
preprocessed_data = '../data/car-price-processed-data.parquet'

In [3]:
#loading data
df = pd.read_parquet(preprocessed_data)

In [4]:
df.dtypes

brand                 object
model                 object
color                 object
transmission_type     object
fuel_type             object
model_age              int64
m_from_car_reg         int64
power_kw               int64
mileage_in_km        float64
price_in_euro          int64
dtype: object

In [5]:
df

,brand,model,color,transmission_type,fuel_type,model_age,m_from_car_reg,power_kw,mileage_in_km,price_in_euro
0,alfa-romeo,Alfa Romeo GTV,red,Manual,Petrol,28,333,148,160500.0,1300
1,alfa-romeo,Alfa Romeo 164,black,Manual,Petrol,28,341,191,190000.0,24900
2,alfa-romeo,Alfa Romeo Spider,black,Unknown,Petrol,28,341,110,129000.0,5900
3,alfa-romeo,Alfa Romeo Spider,black,Manual,Petrol,28,336,110,189500.0,4900
4,alfa-romeo,Alfa Romeo 164,red,Manual,Petrol,27,320,132,96127.0,17950
...,...,...,...,...,...,...,...,...,...,...
251074,volvo,Volvo XC40,white,Automatic,Hybrid,0,3,192,1229.0,57990
251075,volvo,Volvo XC90,white,Automatic,Diesel,0,4,173,4900.0,89690
251076,volvo,Volvo V60,white,Automatic,Diesel,0,2,145,1531.0,61521
251077,volvo,Volvo XC40,white,Automatic,Hybrid,0,2,132,1500.0,57890


In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(df[num_f+cat_f], df[target].values, test_size=0.25, random_state=0)

In [7]:
#creating dummy variables
X_train[cat_f] = X_train[cat_f].astype(str)
train_dict = X_train[cat_f].to_dict(orient = 'records')

dv = DictVectorizer()
X_train_vect = dv.fit_transform(train_dict)

#the same for X_test
X_test[cat_f] = X_test[cat_f].astype(str)
test_dict = X_test[cat_f].to_dict(orient = 'records')
X_test_vect = dv.transform(test_dict)

In [8]:
test_dict

[{'brand': 'volkswagen',
  'model': 'Volkswagen Bus',
  'color': 'white',
  'transmission_type': 'Automatic',
  'fuel_type': 'Diesel'},
 {'brand': 'ford',
  'model': 'Ford Mondeo',
  'color': 'black',
  'transmission_type': 'Automatic',
  'fuel_type': 'Diesel'},
 {'brand': 'opel',
  'model': 'Opel Insignia',
  'color': 'grey',
  'transmission_type': 'Manual',
  'fuel_type': 'Petrol'},
 {'brand': 'audi',
  'model': 'Audi Q5',
  'color': 'black',
  'transmission_type': 'Automatic',
  'fuel_type': 'Diesel'},
 {'brand': 'peugeot',
  'model': 'Peugeot 207',
  'color': 'grey',
  'transmission_type': 'Automatic',
  'fuel_type': 'Petrol'},
 {'brand': 'audi',
  'model': 'Audi A7',
  'color': 'grey',
  'transmission_type': 'Automatic',
  'fuel_type': 'Diesel'},
 {'brand': 'bmw',
  'model': 'BMW 335',
  'color': 'grey',
  'transmission_type': 'Automatic',
  'fuel_type': 'Petrol'},
 {'brand': 'land-rover',
  'model': 'Land Rover Range Rover Evoque',
  'color': 'black',
  'transmission_type': 'Auto

In [9]:
X_train

,model_age,m_from_car_reg,power_kw,mileage_in_km,brand,model,color,transmission_type,fuel_type
206707,7,89,51,99781.0,toyota,Toyota Aygo,red,Manual,Petrol
236858,5,60,110,35000.0,volkswagen,Volkswagen T-Roc,red,Automatic,Petrol
166013,7,82,193,66500.0,porsche,Porsche Cayenne,black,Automatic,Diesel
50938,0,4,67,10.0,dacia,Dacia Sandero,white,Manual,LPG
199388,2,24,150,41000.0,skoda,Skoda Enyaq,grey,Automatic,Electric
...,...,...,...,...,...,...,...,...,...
183100,4,43,55,25996.0,seat,SEAT Mii,white,Manual,Petrol
121727,5,65,180,44100.0,mercedes-benz,Mercedes-Benz E 300,grey,Automatic,Petrol
179819,6,73,132,71000.0,seat,SEAT Leon,white,Automatic,Petrol
45022,5,59,81,45000.0,citroen,Citroen C4 Cactus,grey,Automatic,Petrol


In [10]:
X_train_vect.shape

(183256, 1299)

In [11]:
X_test.shape

(61086, 9)

In [12]:
X_test_vect.shape

(61086, 1299)

In [13]:
# fit model on training data
#model = LinearRegression()
model = XGBRegressor()
model.fit(X_train_vect, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [14]:
Y_train

array([[ 8600],
       [24025],
       [48490],
       ...,
       [19950],
       [17990],
       [ 2500]])

In [15]:
print(model)

from sklearn.metrics import accuracy_score

# make predictions for test data
Y_pred = model.predict(X_train_vect)
# evaluate predictions
#Y_pred = [round(value) for value in Y_pred]
RSME = metrics.mean_squared_error(Y_train, Y_pred, squared = False)
print("RSME train: ", RSME)

# make predictions for test data
Y_pred = model.predict(X_test_vect)
# evaluate predictions
#Y_pred = [round(value) for value in Y_pred]
RSME = metrics.mean_squared_error(Y_test, Y_pred, squared = False)
print("RSME test: ", RSME)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)
RSME train:  10191.806064402273
RSME test:  10308.340508939924


In [16]:
Y_pred

array([26878.326, 21673.162, 14256.604, ..., 10255.493, 18850.113,
       15612.778], dtype=float32)

In [17]:
Y_test

array([[59999],
       [18950],
       [ 5990],
       ...,
       [ 5999],
       [ 3950],
       [23449]])

In [18]:
import pickle
with open('../models/xgb_regressor.bin', 'wb') as f_out:
    pickle.dump((dv, model),f_out)